### Imports

In [1]:
import pandas as pd
import numpy as np
from geolocating_functions import chicago_lat_long_builder, get_community_name
pd.options.mode.chained_assignment = None

### Crimes Dataset

#### Filtering 2019

In [24]:
crimes_full = pd.read_csv('Crimes_-_2001_to_Present.csv')

In [26]:
crimes_full[crimes_full['Year'] == 2019].reset_index(drop=True).to_pickle('crimes_2019.pkl')

In [27]:
crimes_2019 = pd.read_pickle('crimes_2019.pkl')

In [28]:
crimes_2019.columns

#### Isolating columns of choice, drop null

In [29]:
crimes = crimes_2019[['Arrest', 'Primary Type', 'Description', 'Domestic', 'Community Area', 'Location']].dropna().reset_index(drop=True)

crimes.rename(columns={'Arrest': 'arrest', 'Primary Type': 'type', 'Description': 'desc',
                       'Domestic': 'domestic', 'Community Area': 'comm_num', 'Location': 'location'}, inplace=True, errors='ignore')

In [30]:
crimes['location'] = crimes['location'].apply(chicago_lat_long_builder)

#### Convert community number to name

In [31]:
crimes['comm_num'] = crimes['comm_num'].astype(int)

In [32]:
comm_map = pd.read_pickle('comm_map.pkl')

In [33]:
comm_num_to_name_dict = {}

for index, row in comm_map.iterrows():
    comm_num_to_name_dict[row['comm_num']] = row['comm_name']

In [34]:
crimes['community_name'] = crimes['comm_num'].apply(lambda x: comm_num_to_name_dict[x])

In [35]:
crimes.to_pickle('crimes.pkl')

In [3]:
crimes = pd.read_pickle('crimes.pkl')

### Bringing in the rest

#### Adding Community Housing Metrics

In [4]:
community_housing_metrics = pd.read_pickle('community_housing_metrics.pkl')

In [5]:
community_housing_metrics.rename(columns={'Geography': 'community_name', \
                                          'Single Family': 'pct_sfh', \
                                          'Condominium': 'pct_condo', \
                                          'Building with 2-4 Units': 'pct_2_to_4_unit_building', \
                                          'Building with 5+ Units': 'pct_5_plus_unit_building'}, inplace=True, errors='ignore')

In [6]:
crimes_comm_metrics = pd.merge(crimes, community_housing_metrics, on='community_name', how='inner')

#### Merging Crime Radius Metrics

In [7]:
radius_metrics = pd.read_pickle('crime_metrics.pkl')

In [8]:
crimes_comm_radius_metrics = pd.concat([crimes_comm_metrics, round(radius_metrics, 3)], axis=1)

In [9]:
crimes_comm_radius_metrics.fillna(0, inplace=True)

In [11]:
crimes_comm_radius_metrics['housing_market_health'] = (
    (crimes_comm_radius_metrics['total_mortgages'] - crimes_comm_radius_metrics['total_foreclosures']) / crimes_comm_radius_metrics['total_mortgages']
)

In [12]:
crimes_comm_radius_metrics.drop(['total_mortgages', 'total_foreclosures'], axis=1, inplace=True, errors='ignore')

In [13]:
crimes_comm_radius_metrics.to_pickle('crimes_comm_radius_metrics.pkl')

### Building Modeling DataFrame

#### Many columns need slight adjustments

In [7]:
crimes_comm_radius_metrics = pd.read_pickle('crimes_comm_radius_metrics.pkl')

In [8]:
crimes_comm_radius_metrics['type_desc'] = crimes_comm_radius_metrics['type'] + ' ' + crimes_comm_radius_metrics['desc']

In [9]:
crimes_comm_radius_metrics['type_desc'] = crimes_comm_radius_metrics['type_desc'].apply(lambda x: x.lower().replace(' ', '_'))

In [10]:
crimes_comm_radius_metrics['arrest'] = crimes_comm_radius_metrics['arrest'].apply(lambda x: 1 if x == True else 0)

In [11]:
crimes_comm_radius_metrics['domestic'] = crimes_comm_radius_metrics['domestic'].apply(lambda x: 1 if x == True else 0)

In [12]:
crimes_comm_radius_metrics['prestigious_school'] = crimes_comm_radius_metrics['prestigious_school'] * 100

In [13]:
crimes_comm_radius_metrics['pct_grocery_stores_over_10000_sq_ft'] = crimes_comm_radius_metrics['pct_grocery_stores_over_10000_sq_ft'] * 100

In [14]:
crimes_comm_radius_metrics['pct_high_risk_food_establishments'] = crimes_comm_radius_metrics['pct_high_risk_food_establishments'] * 100

In [15]:
crimes_comm_radius_metrics['pct_failed_inspections'] = crimes_comm_radius_metrics['pct_failed_inspections'] * 100

#### Grading 'school quality' based on total scores from survey and growth rating

In [16]:
school_quality_totals = crimes_comm_radius_metrics[['student_growth_rating', 'school_survey_involved_families', 'school_survey_supportive_environment', \
                      'school_survey_ambitious_instruction', 'school_survey_effective_leaders', \
                      'school_survey_collaborative_teachers', 'school_survey_safety']].T.sum()

In [17]:
def school_quality_splits(val):
    
    if val < 20:
        return 'low_avg_school_quality'
    elif (val < 23) & (val >= 20):
        return 'average_school_quality'
    elif (val < 26) & (val >= 23):
        return 'slightly_above_average_school_quality'
    else:
        return 'highly_above_average_school_quality'

In [19]:
crimes_comm_radius_metrics['school_quality'] = school_quality_totals.apply(school_quality_splits)

#### Isolating the crime types to care about

In [20]:
def crime_type_desc(crime_type):
    
    if 'domestic_battery' in crime_type:
        return 'domestic_battery'
    elif 'battery_agg' in crime_type:
        return 'aggravated_battery'
    elif 'battery' in crime_type:
        return 'simple_battery'
    elif 'assault' in crime_type:
        return 'assault'
    elif 'deceptive_practice' in crime_type:
        return 'deceptive_practice'
    elif 'weapons_violation' in crime_type:
        return 'weapons_violation'
    elif 'narcotics' in crime_type:
        return 'narcotics'
    elif 'criminal_damage' in crime_type:
        return 'criminal_damage'
    elif 'criminal_trespass' in crime_type:
        return 'criminal_trespass'
    elif 'burglary' in crime_type:
        return 'burglary'
    elif 'robbery' in crime_type:
        return 'robbery'
    elif 'retail_theft' in crime_type:
        return 'retail_theft'
    elif 'theft' in crime_type:
        return 'theft'
    else:
        return 'other_offense'

In [21]:
crimes_comm_radius_metrics['type_desc'] = crimes_comm_radius_metrics['type_desc'].apply(crime_type_desc)

In [22]:
modeling_crimes_full = crimes_comm_radius_metrics.drop(['type',  'desc', 'comm_num', 'location', 'community_name'], axis=1)

In [44]:
modeling_crimes = pd.concat([modeling_crimes_full.drop([
                            'student_growth_rating', 'school_survey_involved_families',
                            'school_survey_supportive_environment', 'school_survey_ambitious_instruction',
                            'school_survey_effective_leaders', 'school_survey_collaborative_teachers',
                            'school_survey_safety', 'school_quality', 'type_desc'], axis=1),
                             pd.get_dummies(modeling_crimes_full['type_desc']),
                             pd.get_dummies(modeling_crimes_full['school_quality'])], axis=1)

In [45]:
modeling_crimes

,arrest,domestic,pct_sfh,pct_condo,pct_2_to_4_unit_building,pct_5_plus_unit_building,pct_business_share_sales,mobility_rate_pct,chronic_truancy_pct,prestigious_school,...,other_offense,retail_theft,robbery,simple_battery,theft,weapons_violation,average_school_quality,highly_above_average_school_quality,low_avg_school_quality,slightly_above_average_school_quality
0,0,0,63.1,0.0,22.0,14.9,42.5,21.700,16.950,0.0,...,1,0,0,0,0,0,0,0,0,1
1,0,1,63.1,0.0,22.0,14.9,42.5,26.800,62.640,40.0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,63.1,0.0,22.0,14.9,42.5,22.800,53.367,33.3,...,0,0,0,0,0,0,1,0,0,0
3,0,0,63.1,0.0,22.0,14.9,42.5,10.900,42.300,50.0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,63.1,0.0,22.0,14.9,42.5,18.667,39.967,33.3,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258609,0,0,69.4,0.5,21.9,8.2,6.5,42.633,76.567,33.3,...,0,0,0,0,0,0,0,0,1,0
258610,0,0,69.4,0.5,21.9,8.2,6.5,10.000,23.300,0.0,...,0,0,0,0,1,0,0,0,1,0
258611,0,0,69.4,0.5,21.9,8.2,6.5,24.950,48.963,50.0,...,0,1,0,0,0,0,1,0,0,0
258612,0,0,69.4,0.5,21.9,8.2,6.5,17.783,27.780,50.0,...,0,0,0,0,0,0,1,0,0,0


In [46]:
modeling_crimes.to_pickle('modeling_crimes.pkl')

### Building Tableau Dataframes

#### Crimes w/ metrics

In [34]:
crimes_for_tableau = crimes_comm_radius_metrics.copy()

In [35]:
crimes_for_tableau['lat'] = crimes_for_tableau['location'].apply(lambda x: x[0])

In [36]:
crimes_for_tableau['long'] = crimes_for_tableau['location'].apply(lambda x: x[1])

In [37]:
crimes_for_tableau.drop(['type', 'desc', 'comm_num', 'location'], axis=1, inplace=True, errors='ignore')

In [38]:
crimes_for_tableau = pd.concat([crimes_for_tableau.drop([
                            'student_growth_rating', 'school_survey_involved_families',
                            'school_survey_supportive_environment', 'school_survey_ambitious_instruction',
                            'school_survey_effective_leaders', 'school_survey_collaborative_teachers',
                            'school_survey_safety', 'school_quality'], axis=1),
                             pd.get_dummies(crimes_for_tableau['school_quality'])], axis=1)

In [39]:
crimes_for_tableau.to_pickle('crimes_for_tableau.pkl')

#### Clinics

In [7]:
clinics = pd.read_pickle('clinics.pkl')

In [8]:
clinics['lat'] = clinics['location'].apply(lambda x: x[0])

In [9]:
clinics['long'] = clinics['location'].apply(lambda x: x[1])

In [10]:
clinics_for_tableau = clinics[['facility', 'community_name', 'lat', 'long']]

In [11]:
clinics_for_tableau['facility_type'] = pd.Series('clinic', index=clinics_for_tableau.index)

In [12]:
clinics_for_tableau.rename(columns={'facility': 'name'}, inplace=True, errors='ignore')

#### Grocery Stores

In [13]:
grocery_stores = pd.read_pickle('grocery_stores.pkl')

In [14]:
grocery_stores['lat'] = grocery_stores['location'].apply(lambda x: x[0])

In [15]:
grocery_stores['long'] = grocery_stores['location'].apply(lambda x: x[1])

In [16]:
grocery_stores_for_tableau = grocery_stores[['store_name', 'community_name', 'lat', 'long']]

In [17]:
grocery_stores_for_tableau['facility_type'] = pd.Series('grocery_store', index=grocery_stores_for_tableau.index)

In [18]:
grocery_stores_for_tableau.rename(columns={'store_name': 'name'}, inplace=True, errors='ignore')

#### Libraries

In [19]:
libraries = pd.read_pickle('libraries.pkl')

In [20]:
libraries['lat'] = libraries['location'].apply(lambda x: x[0])

In [21]:
libraries['long'] = libraries['location'].apply(lambda x: x[1])

In [22]:
libraries_for_tableau = libraries[['name', 'community_name', 'lat', 'long']]

In [23]:
libraries_for_tableau['facility_type'] = pd.Series('library', index=libraries_for_tableau.index)

#### Pharmacies

In [24]:
pharmacies = pd.read_pickle('pharmacies.pkl')

In [25]:
pharmacies['lat'] = pharmacies['location'].apply(lambda x: x[0])

In [26]:
pharmacies['long'] = pharmacies['location'].apply(lambda x: x[1])

In [27]:
pharmacies_for_tableau = pharmacies[['pharmacy_name', 'community_name', 'lat', 'long']]

In [28]:
pharmacies_for_tableau['facility_type'] = pd.Series('pharmacy', index=pharmacies_for_tableau.index)

In [29]:
pharmacies_for_tableau.rename(columns={'pharmacy_name': 'name'}, inplace=True, errors='ignore')

#### Schools

In [30]:
schools = pd.read_pickle('schools.pkl')

In [31]:
schools['lat'] = schools['location'].apply(lambda x: x[0])

In [32]:
schools['long'] = schools['location'].apply(lambda x: x[1])

In [33]:
schools_for_tableau = schools[['long_name', 'community_name', 'lat', 'long']]

In [34]:
schools_for_tableau['facility_type'] = pd.Series('school', index=schools_for_tableau.index)

In [35]:
schools_for_tableau.rename(columns={'long_name': 'name'}, inplace=True, errors='ignore')

#### Combining Facilities

In [36]:
facilities_for_tableau = pd.concat([clinics_for_tableau, grocery_stores_for_tableau, libraries_for_tableau, pharmacies_for_tableau, schools_for_tableau])

In [39]:
facilities_for_tableau['facility_type'] = facilities_for_tableau['facility_type'].apply(lambda x: x.replace('_', ' ').title())

In [40]:
facilities_for_tableau.to_csv('facilities_for_tableau.csv')